In [ ]:
# | default_exp cli

In [ ]:
# | export

from typing import *
import os
import re

import typer
import pathlib

from faststream_gen._components.logger import get_logger
from faststream_gen._code_generator.app_description_validator import validate_app_description
from faststream_gen._code_generator.asyncapi_spec_generator import generate_asyncapi_spec
from faststream_gen._code_generator.app_generator import generate_app
from faststream_gen._code_generator.test_generator import generate_test
from faststream_gen._code_generator.helper import set_logger_level, add_tokens_usage, write_file_contents
from faststream_gen._code_generator.constants import DEFAULT_MODEL, MODEL_PRICING, TOKEN_TYPES, DESCRIPTION_FILE_NAME, \
                                                    GENERATE_APP_FROM_ASYNCAPI, GENERATE_APP_FROM_SKELETON, GENERATE_APP_SKELETON

In [ ]:
from typer.testing import CliRunner
import pytest
from unittest.mock import patch
from tempfile import TemporaryDirectory

from faststream_gen._components.logger import suppress_timestamps

In [ ]:
# | export

logger = get_logger(__name__)

In [ ]:
suppress_timestamps()
logger = get_logger(__name__, level=20)
logger.info("ok")

[INFO] __main__: ok


In [ ]:
runner = CliRunner()

In [ ]:
# | export

OPENAI_KEY_EMPTY_ERROR = "Error: OPENAI_API_KEY cannot be empty. Please set a valid OpenAI API key in OPENAI_API_KEY environment variable and try again.\nYou can generate API keys in the OpenAI web interface. See https://platform.openai.com/account/api-keys for details."
OPENAI_KEY_NOT_SET_ERROR = "Error: OPENAI_API_KEY not found in environment variables. Set a valid OpenAI API key in OPENAI_API_KEY environment variable and try again. You can generate API keys in the OpenAI web interface. See https://platform.openai.com/account/api-keys for details."


def _ensure_openai_api_key_set() -> None:
    """Ensure the 'OPENAI_API_KEY' environment variable is set and is not empty.

    Raises:
        KeyError: If the 'OPENAI_API_KEY' environment variable is not found.
        ValueError: If the 'OPENAI_API_KEY' environment variable is found but its value is empty.
    """
    try:
        openai_api_key = os.environ["OPENAI_API_KEY"]
        if openai_api_key == "":
            raise ValueError(OPENAI_KEY_EMPTY_ERROR)
    except KeyError:
        raise KeyError(OPENAI_KEY_NOT_SET_ERROR)

In [ ]:
with patch.dict(os.environ, {"OPENAI_API_KEY": ""}):
    with pytest.raises(ValueError) as e:
        _ensure_openai_api_key_set()

print(e.value)
assert str(e.value) == OPENAI_KEY_EMPTY_ERROR

Error: OPENAI_API_KEY cannot be empty. Please set a valid OpenAI API key in OPENAI_API_KEY environment variable and try again.
You can generate API keys in the OpenAI web interface. See https://platform.openai.com/account/api-keys for details.


In [ ]:
with patch.dict(os.environ, {}, clear=True):
    with pytest.raises(KeyError) as e:
        _ensure_openai_api_key_set()
        
print(e.value)
assert str(e.value) == f"'{OPENAI_KEY_NOT_SET_ERROR}'"

'Error: OPENAI_API_KEY not found in environment variables. Set a valid OpenAI API key in OPENAI_API_KEY environment variable and try again. You can generate API keys in the OpenAI web interface. See https://platform.openai.com/account/api-keys for details.'


In [ ]:
with patch.dict(os.environ, {"OPENAI_API_KEY": "INVALID_KEY"}):
    _ensure_openai_api_key_set()

In [ ]:
# | export

app = typer.Typer(
    short_help="Commands for accelerating FastStream app creation using advanced AI technology",
     help="""Commands for accelerating FastStream app creation using advanced AI technology.

These commands use a combination of OpenAI's gpt-3.5-turbo and gpt-3.5-turbo-16k models to generate FastStream code. To access this feature, kindly sign up if you haven't already and create an API key with OpenAI. You can generate API keys in the OpenAI web interface. See https://platform.openai.com/account/api-keys for details.

Once you have the key, please set it in the OPENAI_API_KEY environment variable before executing the code generation commands.

Note: Accessing OpenAI API incurs charges. However, when you sign up for the first time, you usually get free credits that are more than enough to generate multiple FastStream apps. For further information on pricing and free credicts, check this link: https://openai.com/pricing
    """,
)

In [ ]:
# | export


def _strip_white_spaces(description: str) -> str:
    """Remove and strip excess whitespaces from a given description

    Args:
        description: The description string to be processed.

    Returns:
        The cleaned description string.
    """
    pattern = re.compile(r"\s+")
    return pattern.sub(" ", description).strip()

In [ ]:
fixture = """
    I have   a                  lot
                of whitespaces
                
                
"""

expected = "I have a lot of whitespaces"
actual = _strip_white_spaces(fixture)
print(actual)
assert actual == expected

I have a lot of whitespaces


In [ ]:
# | export


def _calculate_price(total_tokens_usage: Dict[str, int], model: str = DEFAULT_MODEL) -> float:
    """Calculates the total price based on the number of promt & completion tokens and the models price for input and output tokens (per 1k tokens).

    Args:
        total_tokens_usage: OpenAI "usage" dictionaries which defines prompt_tokens, completion_tokens and total_tokens


    Returns:
        float: The price for used tokens
    """
    model_price = MODEL_PRICING[model]
    price = (total_tokens_usage["prompt_tokens"] * model_price["input"] + total_tokens_usage["completion_tokens"] * model_price["output"]) / 1000
    return price

In [ ]:
usage = {
    "prompt_tokens": 129,
    "completion_tokens": 1,
    "total_tokens": 130
  }

usage_list = [usage, usage]
total_tokens_usage = add_tokens_usage(usage_list)

assert _calculate_price(total_tokens_usage) == 0.000782

In [ ]:
# | export

EMPTY_DESCRIPTION_ERROR = "Error: you need to provide the application description by providing it with the command line argument or by providing it within a textual file wit the --input_file argument."

def _get_description(input_path: str) -> str:
    """Reads description from te file and returns it as a string

    Args:
        input_path: Path to the file with the desription
        
    Raises:
        ValueError: If the file does not exist.

    Returns:
        The description string which was read from the file.
    """
    try:
        with open(input_path) as file:
            # Read all lines to list
            lines = file.readlines()
            # Join the lines 
            description = '\r'.join(lines)
            logger.info(f"Reading application description from '{str(pathlib.Path(input_path).absolute())}'.")
    except Exception as e:
        raise ValueError(f"Error while reading from the file: '{str(pathlib.Path(input_path).absolute())}'\n{str(e)}")
    return description

In [ ]:
with TemporaryDirectory() as d:
    input_path = f"{d}/app_description.txt"
    description = """Create a FastKafka application using localhost broker for testing, staging.airt.ai for staging and prod.airt.ai for production. Use default port number.

It should consume from 'store_product' topic an JSON encoded object with the following three attributes: product_name, currency and price. The format of the currency will be three letter string, e.g. 'EUR'.
For each consumed message, check if the currency attribute is set to 'HRK'. If it is then change the currency to 'EUR' and divide the price by 7.5, if the currency is not set to 'HRK' don't change the original message. Finally, publish the consumed message to 'change_currency' topic.

Use SASL_SSL with SCRAM-SHA-256 for authentication with username and password."""
    
    
    write_file_contents(input_path, description)
    actual = _get_description(input_path)
    display(actual)
    expected = "Create a FastKafka application using localhost broker for testing, staging.airt.ai for staging and prod.airt.ai for production. Use default port number.\n\r\n\rIt should consume from 'store_product' topic an JSON encoded object with the following three attributes: product_name, currency and price. The format of the currency will be three letter string, e.g. 'EUR'.\n\rFor each consumed message, check if the currency attribute is set to 'HRK'. If it is then change the currency to 'EUR' and divide the price by 7.5, if the currency is not set to 'HRK' don't change the original message. Finally, publish the consumed message to 'change_currency' topic.\n\r\n\rUse SASL_SSL with SCRAM-SHA-256 for authentication with username and password."         
    
    assert actual == expected, actual

[INFO] __main__: Reading application description from '/tmp/tmpu5rnou5l/app_description.txt'.


"Create a FastKafka application using localhost broker for testing, staging.airt.ai for staging and prod.airt.ai for production. Use default port number.\n\r\n\rIt should consume from 'store_product' topic an JSON encoded object with the following three attributes: product_name, currency and price. The format of the currency will be three letter string, e.g. 'EUR'.\n\rFor each consumed message, check if the currency attribute is set to 'HRK'. If it is then change the currency to 'EUR' and divide the price by 7.5, if the currency is not set to 'HRK' don't change the original message. Finally, publish the consumed message to 'change_currency' topic.\n\r\n\rUse SASL_SSL with SCRAM-SHA-256 for authentication with username and password."

In [ ]:
with pytest.raises(ValueError) as e:
        _get_description("incorrect_path")
        
print(e.value)
assert str(e.value) == f"Error while reading from the file: '{str(pathlib.Path('incorrect_path').absolute())}'\n[Errno 2] No such file or directory: 'incorrect_path'"

Error while reading from the file: '/work/fastkafka-gen/nbs/incorrect_path'
[Errno 2] No such file or directory: 'incorrect_path'


In [ ]:
# | export


@app.command(
    "generate",
    help="Effortlessly generate FastStream application code and integration tests from the app description.",
)
@set_logger_level
def generate_fastkafka_app(
    description: Optional[str] = typer.Argument(
        None,
        help="""Summarize your FastStream app in a few sentences!


\nInclude details about messages, topics, servers, and a brief overview of the intended business logic.


\nThe simpler and more specific the app description is, the better the generated app will be. Please refer to the below example for inspiration:


\nCreate a FastStream application using localhost broker for testing and use the default port number. 
It should consume messages from the "input_data" topic, where each message is a JSON encoded object containing a single attribute: 'data'. 
For each consumed message, create a new message object and increment the value of the data attribute by 1. Finally, send the modified message to the 'output_data' topic.
\n"""
    ),
    input_path: str = typer.Option(
        None,
        "--input_file",
        "-i",
        help="""
        The path to the file with the app desription. This path should be relative to the current working directory.
        
        \n\nIf the app description is passed via both a --input_file and a command line argument, the description from the command line will be used to create the application.
        """,
    ),
    output_path: str = typer.Option(
        "./faststream-gen",
        "--output_path",
        "-o",
        help="The path to the output directory where the generated files will be saved. This path should be relative to the current working directory.",
    ),
    verbose: bool = typer.Option(
        False,
        "--verbose",
        "-v",
        help="Enable verbose logging by setting the logger level to INFO.",
    ),
) -> None:
    """Effortlessly generate an AsyncAPI specification, FastStream application code, and integration tests from the app description."""
    try:
        tokens_list: List[Dict[str, int]] = []
        _ensure_openai_api_key_set()
        if not description:
            if not input_path:
                raise ValueError(EMPTY_DESCRIPTION_ERROR)             
            description = _get_description(input_path)
        
        cleaned_description = _strip_white_spaces(description)
        validated_description, tokens_list = validate_app_description(cleaned_description, tokens_list)
        write_file_contents(f"{output_path}/{DESCRIPTION_FILE_NAME}", validated_description)
        
#         tokens_list = generate_asyncapi_spec(validated_description, output_path, tokens_list)
#         tokens_list = generate_app(output_path, tokens_list, GENERATE_APP_FROM_ASYNCAPI)      
        tokens_list = generate_app(output_path, tokens_list, GENERATE_APP_SKELETON)
        tokens_list = generate_app(output_path, tokens_list, GENERATE_APP_FROM_SKELETON)
        
        tokens_list = generate_test(validated_description, output_path, tokens_list)
        
        fg = typer.colors.CYAN
    except (ValueError, KeyError) as e:
        fg = typer.colors.RED
        typer.secho(e, err=True, fg=fg)
        raise typer.Exit(code=1)
    except Exception as e:
        fg = typer.colors.RED
        typer.secho(f"Unexpected internal error: {e}", err=True, fg=fg)
        raise typer.Exit(code=1)
    finally:
        total_tokens_usage = add_tokens_usage(tokens_list)
        price = _calculate_price(total_tokens_usage)
        
        typer.secho(f" Tokens used: {total_tokens_usage['total_tokens']}", fg=fg)
        logger.info(f"Prompt Tokens: {total_tokens_usage['prompt_tokens']}")
        logger.info(f"Completion Tokens: {total_tokens_usage['completion_tokens']}")
        typer.secho(f" Total Cost (USD): ${round(price, 5)}", fg=fg)
        
        phases = ["validation", "specification generation", "app generation", "test generation"]
        logger.info("Number of tokens per phase:")
        for i, token in enumerate(tokens_list):
            logger.info(f"{phases[i]}: {token['total_tokens']} tokens")
        
    typer.secho("✨  All files were successfully generated!", fg=fg)
        

In [ ]:
# | notest

! nbdev_export

In [ ]:
result = runner.invoke(app, ["generate", "--help"])

                                                                                                                   
 Usage: generate [OPTIONS] [DESCRIPTION]                                                                           
                                                                                                                   

Effortlessly generate FastStream application code and integration tests from the app description.

╭─ Arguments ─────────────────────────────────────────────────────────────────────────────────────────────────────╮
│   description      [DESCRIPTION]  Summarize your FastStream app in a few sentences!                             │
│                                                                                                                 │
│                                   Include details about messages, topics, servers, and a brief overview of the  │
│                                   intended business logic.                                                      │
│                                                                                                                 │
│                                   The simpler and more specific the app description is, the better the          │
│                                   generated app will be. Please refer to the below example for inspiration:     │
│                                                                                                                 │
│                                   Create a FastStream application using localhost broker for testing and use    │
│                                   the default port number.  It should consume messages from the "input_data"    │
│                                   topic, where each message is a JSON encoded object containing a single        │
│                                   attribute: 'data'.  For each consumed message, create a new message object    │
│                                   and increment the value of the data attribute by 1. Finally, send the         │
│                                   modified message to the 'output_data' topic.                                  │
│                                   [default: None]                                                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ Options ───────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ --input_file          -i      TEXT  The path to the file with the app desription. This path should be relative  │
│                                     to the current working directory.                                           │
│                                     If the app description is passed via both a --input_file and a command line │
│                                     argument, the description from the command line will be used to create the  │
│                                     application.                                                                │
│                                     [default: None]                                                             │
│ --output_path         -o      TEXT  The path to the output directory where the generated files will be saved.   │
│                                     This path should be relative to the current working directory.              │
│                                     [default: ./faststream-gen]                                                 │
│ --verbose             -v            Enable verbose logging by setting the logger level to INFO.                 │
│ --install-completion                Install completion for the current shell.                                   │
│ --show-completion                   Show completion for the current shell, to copy it or customize the          │
│                                     installation.                                                               │
│ --help                              Show this message and exit.                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯